In [8]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

from os import path
import sys
sys.path.append(path.join(".."))

import saly
import saly.backend as S
import pandas as pd
import numpy as np

%load_ext autoreload
%autoreload 2

EPOCHS = 1
SPECIES = 'Mouse'
DATA_PATH = '/home/mlongar/Data/SingleCellGeneExpression/'
DATASET_HRVATIN_PATH = DATA_PATH + 'hrvatin_2018.h5ad' 
DATASET_CHEN_PATH = DATA_PATH + 'chen_2017.h5ad'
DATASET_BIG_PATH = DATA_PATH + '10x_mouse_1.3mln.h5ad'
MARKER_PATH = DATA_PATH + 'panglao_gene_markers.tab.gz'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
hrvatin_data = saly.load_h5ad(DATASET_HRVATIN_PATH)
chen_data = saly.load_h5ad(DATASET_CHEN_PATH)
#big_data = saly.load_h5ad(DATASET_BIG_PATH)

print("HRVATIN DATA\n", hrvatin_data.shape)
print("CHEN DATA\n", chen_data.shape)
# print("BIG DATA\n", big_data.shape)

HRVATIN DATA
 (48266, 25186)
CHEN DATA
 (14437, 23284)


In [10]:
chen_data = saly.drop_rows(chen_data, ['tanycyte', 'unknown', 'ependymal cell', 'epithelial cell'])

Dropped 4750 cell(s). New shape: (9687, 23284)


In [11]:
markers_db = saly.load_markers(MARKER_PATH, SPECIES)
markers = saly.get_mutual_markers([hrvatin_data, chen_data], markers_db)

print('Mutual markers:', len(markers))

Mutual markers: 7388


In [12]:
marker_aliases = {'Neuron' : 'Neurons',
                'Oligodendrocyte' : 'Oligodendrocytes',
                'Muscle cell' : 'Smooth muscle cells',
                'Endothelial cell' : 'Endothelial cells',
                'Astrocyte' : 'Astrocytes',
                'Macrophage' : 'Macrophages',
                'OPC' : 'Oligodendrocyte progenitor cells',
                'Pericyte' : 'Pericytes'}

saly.check_labels([hrvatin_data, chen_data], markers, marker_aliases)

In [13]:
hrvatin_data = saly.drop_unused_genes(hrvatin_data, markers)
chen_data = saly.drop_unused_genes(chen_data, markers)

saly.check_shape([hrvatin_data, chen_data])

Dropped 21104 gene(s). New shape: (48266, 4082)
Dropped 19202 gene(s). New shape: (9687, 4082)


In [ ]:
train_data, _, _ = saly.preprocess_data(hrvatin_data, splits=4,
                                        train=1.0, validation=0.0, test=0.0)

Split 0
Column 0
Column 500
Column 1000
Column 1500
Column 2000
Column 2500
Column 3000
Column 3500
Column 4000


Trying to set attribute `.X` of view, making a copy.


Split 1
Column 0
Column 500
Column 1000
Column 1500
Column 2000
Column 2500
Column 3000
Column 3500
Column 4000
Split 2
Column 0
Column 500
Column 1000
Column 1500
Column 2000
Column 2500
Column 3000
Column 3500
Column 4000


In [ ]:
model, marker_model, encoder_model = saly.build_model(big_data, markers, supervised=False);